In [1]:
import argparse
import glob
from hydra import compose, initialize
import hydra
import numpy as np
import pandas as pd
import torch
import yaml
from omegaconf import DictConfig, OmegaConf
from dl4cv.utils.utils import set_seed
from dl4cv.lightning_classes.plmodel import LitODModel
from dl4cv.datasets import build_taco 
from dl4cv.datasets.taco_data import taco_val_test_collate_fn
from dl4cv.utils.object_detect_utils import get_iou, fix_orientation, mAP_calc, calc_total_mAP
from tqdm import tqdm
import os


In [2]:
hydra.core.global_hydra.GlobalHydra.instance().clear()
initialize(config_path="./configs")
inference_cfg = compose(config_name="config_taco_training")
inference_cfg["inference"]["run_name"] = 'Resnet50_Whale_bs1'
inference_cfg["inference"]["device"] = 'cuda:1'
path = f"outputs/{inference_cfg.inference.run_name}/.hydra/config.yaml"
with open(path) as cfg:
    cfg_yaml = yaml.safe_load(cfg)
cfg_yaml["inference"] = inference_cfg["inference"]
cfg_yaml["datamodule"]["test"]["params"]["num_to_return"] = 100
cfg = OmegaConf.create(cfg_yaml)


In [3]:
def main(cfg: DictConfig) -> None:
    """
    Run pytorch-lightning model inference
    Args:
        cfg: hydra config
    Returns:
        None
    """
    set_seed(cfg.training.seed)

    device = torch.device(cfg.inference.device)

    model_names = glob.glob(f"outputs/{cfg.inference.run_name}/saved_models/*.ckpt")
    _, _, test_set,_ = build_taco(cfg)
    test_set.num_to_return = 100
    # Dirty trick to get the ground truth boxes
    loader = torch.utils.data.DataLoader(
        test_set,
        collate_fn=taco_val_test_collate_fn,
        batch_size=1,
        num_workers=1,
        shuffle=False,
    )
    lit_model = LitODModel.load_from_checkpoint(checkpoint_path=model_names[0], cfg=cfg)
    lit_model.to(device)
    predictions = []
    for batch in tqdm(loader):
        # move batch elements to device
        batch = tuple(b.to(device) for b in batch)
        predictions.append(
            lit_model.nms_on_image(
                batch,
            )
        )

    return predictions




In [4]:
predictions = main(cfg)

In [ ]:
predictions[0].keys()

In [ ]:
# find an index in predictions where the kept_preds are not empty
valids = []
for i in range(len(predictions)):
    if len(predictions[i]['kept_preds']) > 0:
        valids.append(i)

In [ ]:
predictions[4]

In [ ]:
pred = []
truth = []

for i in range(len(predictions)):
    if len(predictions[i]['kept_preds']) > 0:
        pred.append(predictions[i]['kept_preds'])
        _gt_boxes = predictions[i]['gt_regions']
        _gt_classes = predictions[i]['gt_labels']
        _truth = []
        for j in range(len(_gt_boxes)):
            _box = {'x1':_gt_boxes[j][0].item(), 'x2':_gt_boxes[j][1].item(), 'y1':_gt_boxes[j][2].item(), 'y2':_gt_boxes[j][3].item()}
            _class = _gt_classes[j].item()
            _truth.append({'bbox':_box, 'pred_class':_class})
        truth.append(_truth)

In [ ]:
calc_total_mAP(pred,truth)

In [ ]:
mAP_calc(pred,truth)

In [ ]:
calc_total_mAP(pred,truth) 

In [ ]:
mAP_calc(pred,truth)